In [ ]:
%matplotlib inline


# Robust OPT - Sampling - Rosenbrock function


In [ ]:
from __future__ import annotations

from gemseo import configure_logger
from gemseo.algos.design_space import DesignSpace
from gemseo.algos.parameter_space import ParameterSpace
from gemseo.disciplines.analytic import AnalyticDiscipline

from gemseo_umdo.scenarios.umdo_scenario import UMDOScenario

configure_logger()

Firstly,
we define an [AnalyticDiscipline][gemseo.disciplines.analytic.AnalyticDiscipline]
implementing a random version of the Rosenbrock function
$f(x,y,U)=(U-x)^2+100(y-x^2)^2$:



In [ ]:
discipline = AnalyticDiscipline({"z": "(u-x)**2+100*(y-x**2)**2"}, name="Rosenbrock")

where $x,y$ belongs to the interval $[-2,2]$:



In [ ]:
design_space = DesignSpace()
design_space.add_variable("x", l_b=-2, u_b=2.0, value=-2.0)
design_space.add_variable("y", l_b=-2, u_b=2.0, value=-2.0)

and $U$ is a Gaussian variable with unit mean
and standard deviation equal to 0.05:



In [ ]:
uncertain_space = ParameterSpace()
uncertain_space.add_random_variable("u", "OTNormalDistribution", mu=1.0, sigma=0.05)

Then,
we build a [UMDOScenario][gemseo_umdo.scenarios.umdo_scenario.UMDOScenario]
to minimize a sampling-based estimation
of the expectation $\mathbb{E}[Y]$ where $Y=f(x,y,U)$:



In [ ]:
scenario = UMDOScenario(
    [discipline],
    "DisciplinaryOpt",
    "z",
    design_space,
    uncertain_space,
    "Mean",
    statistic_estimation="Sampling",
    statistic_estimation_parameters={"n_samples": 10},
)

and execute it with a gradient-based optimizer:



In [ ]:
scenario.set_differentiation_method("finite_differences")

.. note::
   The statistics do not allow for the moment to use analytical derivatives.
   Please use finite differences or complex step to approximate the gradients.



In [ ]:
scenario.execute({"algo": "NLOPT_SLSQP", "max_iter": 100})

Lastly,
we can plot the optimization history view:



In [ ]:
scenario.post_process("OptHistoryView", save=True, show=True)